# 最优游戏策略： 扑克牌抽取

## 问题描述

100张扑克牌，50红、50黑。玩家参与一个游戏：每回合随机抽一张牌，如果是红色，玩家得到一元，如果是黑色，玩家损失一元，已经抽过的牌不放回牌库。每一个回合结束的时候玩家可以选择终止游戏。问为了最大化收益，玩家应该采取怎样的策略结束游戏，该策略下预期收益是多少。

## 数学模型

设$x_i$为第$i$张牌抽取后的收益，其中
$$
    x_i = \left\{ 
        \begin{aligned}
            1, & \text{如果第$i$次抽取的是红色,} \\
            -1, & \text{如果第$i$次抽取的是黑色.} \\
        \end{aligned}
    \right.
$$




则n次随机抽取后的收益为
$$
\sum_{i=1}^n x_i, 1\leq n \leq 100.
$$

我们要优化的目标函数为
$$
    \omega(s) = \max_{n\in[1,100]} \operatorname{E}\left(\sum_{i=1}^n x_i\right).
$$

<!-- 目标函数描述存在问题 -->

## 常见策略

1. 静态张数：不论抽到什么牌，抽n张就结束。
2. 静态收益: 累计收益n元结束，如果一直没达到就全部抽完。
3. 动态规划： 对于每个已出现(或剩余)的x红+y黑的情况都通过某一个准则决定是否继续抽，这种准则是预先确定的，整个决策的期望值是唯一的。

## 1. 静态张数

这种策略的期望收益我们可以通过数学计算直接得出：

对于任意的n, 
$$
\begin{aligned}
    &\operatorname{E}\left(\sum_{i=1}^n x_i\right) \\
    =&\sum_{i=1}^n \operatorname{E}\left(x_i\right) \\ 
    =&\sum_{i=1}^n \frac{1}{2} \cdot 1 + \frac{1}{2} \cdot (-1)\\
    =& 0.
\end{aligned}
$$

## 2. 静态收益

In [13]:
import random
import numpy as np

targetRevenue = 4
nowRevenue = 0
redcard = 50
blackcard = 50

Revenues = []
iterNum = 100000
for i in range(iterNum):

    nowRevenue = 0
    redcard = 50
    blackcard = 50

    while nowRevenue < targetRevenue and redcard > 0 and blackcard > 0:
        temp = random.choice([1,-1])
        if temp == 1:
            redcard -= 1
        else:
            blackcard -= 1
        nowRevenue += temp

    Revenues.append(nowRevenue)

print(np.average(Revenues)) 


-0.01072


从静态收益的模拟结果看，平均收益在0左右，该策略的收益并不理想

## 3. 动态规划

我们可以定义状态 (dp[i][j]) 表示剩下 (i) 张红色牌和 (j) 张黑色牌时，遵循最优策略能够获得的最大期望收益。我们需要找到状态转移方程，并计算 (dp[50][50]) 的值。

### 状态转移方程
对于任意状态 (dp[i][j])，玩家面临的选择是继续或停止：

如果继续，玩家抽到红色牌的概率是 $\frac{i}{i+j}$，抽到黑色牌的概率是 $\frac{j}{i+j}$。因此，如果继续抽牌，期望收益是 $\frac{i}{i+j} \cdot (dp[i-1][j] + 1) + \frac{j}{i+j} \cdot (dp[i][j-1] - 1)$。
如果停止，期望收益是 (0)（因为不再有收益或损失）。
最优策略的期望收益是上述两种选择的最大值。因此，我们有：

$$dp[i][j] = \max\left(0, \frac{i}{i+j} \cdot (dp[i-1][j] + 1) + \frac{j}{i+j} \cdot (dp[i][j-1] - 1)\right)$$

### 边界条件
当没有红色或黑色牌剩余时，即 (i = 0) 或 (j = 0)，游戏应该停止，因为无法再获得正的期望收益。因此，对于所有 (i)，(dp[i][0] = 0)；对于所有 (j)，(dp[0][j] = 0)。
接下来，我们可以编写代码来实现这个动态规划解决方案，并计算 (dp[50][50]) 的值，即初始状态下遵循最优策略的最大期望收益。

In [1]:
import numpy as np

# 初始化dp数组，所有元素设为None，表示未计算
dp = np.full((51, 51), None)

# 边界条件
for i in range(51):
    dp[i][0] = 0
    dp[0][i] = 0

# 动态规划计算dp值
for i in range(1, 51):
    for j in range(1, 51):
        continue_expectation = (i / (i + j)) * (dp[i-1][j] + 1) + (j / (i + j)) * (dp[i][j-1] - 1)
        dp[i][j] = max(0, continue_expectation)

# dp[50][50]是初始状态下最大期望收益
dp[50][50]


3.1986936977776326

据动态规划的计算结果，如果玩家遵循最优策略，即在每一步都根据当前剩余的红色和黑色牌计算是否继续或停止来最大化期望收益，他们的最大期望收益大约为 3.20 元。

因此，最优策略应该是基于当前牌的组合动态决定是否继续游戏。具体地，玩家应该在期望收益为正时继续游戏，并在期望收益不再为正时停止游戏。这种策略确保了玩家在给定的牌组合下获得最大的期望收益